<img src = "https://i.imgur.com/UjutVJd.jpg" align = "center">

## Vanishing Gradient Problem

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=7)

Seperti yang telah dijelaskan, standar Jaringan Saraf Tiruan mengalami beberapa permasalahan yang menyebabkan Jaringan tidak bisa diperdalam. Salah satunya adalah permasalahan *Vanishing Gradient Problem*,

Untuk mendalami permasalahan ini, mula-mula mari kita buat kembali implementasi API Neural Net seperti sebelumnya

In [0]:
def affine_forward(x, W, b):   
  
    v = np.dot(x, W) + b # x dot w + b
    
    cache = (x, W, b)
    
    return v, cache

In [0]:
def affine_backward(dout, cache):
    
    x, W, b = cache
    
    dW = np.dot(x.T, dout) # x' dot dout
    
    db = np.sum(dout, axis=0, keepdims=True)
    
    dx = np.dot(dout, W.T) # dout dot W'
    
    return dW, db, dx

In [0]:
def sigmoid_forward(x):  
  
    out = 1 / ( 1 + np.exp(-x) ) 
    
    return out  

In [0]:
def sigmoid_backward(dout, ds):
    """
    Argument:
        ds: sigmoid forward result
        dout: gradient error
    """
    ds_ = ds * ( 1 - ds )
    
    dout = dout * ds_
    
    return dout

### Vanishing Gradient di Fungsi Aktivasi Sigmoid

Fungsi Aktivasi Sigmoid dan Tanh adalah fungsi aktivasi yang hampir selalu digunakan pada awal perkembangan Jaringan Saraf dikarenakan memiliki intepretasi dan intusisi yang pas mengenai *firing rate* dari neuron. Sigmoid menekan output ke dalam rentang $[0..1]$ (dan rentang $[-1..1]$ untuk tanh). 


*Sigmoid function* | *Tanh function*
-- | --
![sigmoid](http://cs231n.github.io/assets/nn1/sigmoid.jpeg) | ![tanh](http://cs231n.github.io/assets/nn1/tanh.jpeg)


Proses tersebut menyebabkan makin tinggi input sigmoid maka makin mendekati nilai 1 outputnya dengan presisi desimal yang sangat tinggi. Dan hal ini akan menyebabkan nilai gradien yang dikembalikan juga semakin kecil.

Perhatikan contoh di bawah

In [9]:
print('sigmoid(.01) =',sigmoid_forward(.01))
print('sigmoid(.1)  =',sigmoid_forward(.1))
print('sigmoid(.5)  =',sigmoid_forward(.5))
print('sigmoid(1)   =',sigmoid_forward(1))
print('sigmoid(10)  =',sigmoid_forward(10))
print('sigmoid(50)  =',sigmoid_forward(50))

sigmoid(.01) = 0.5024999791668749
sigmoid(.1)  = 0.52497918747894
sigmoid(.5)  = 0.6224593312018546
sigmoid(1)   = 0.7310585786300049
sigmoid(10)  = 0.9999546021312976
sigmoid(50)  = 1.0


In [10]:
print('sigmoid backward(50)  =',sigmoid_backward(1,sigmoid_forward(50)))
print('sigmoid backward(10)  =',sigmoid_backward(1,sigmoid_forward(10)))
print('sigmoid backward(1)   =',sigmoid_backward(1,sigmoid_forward(1)))
print('sigmoid backward(0.5) =',sigmoid_backward(1,sigmoid_forward(.5)))

sigmoid backward(50)  = 0.0
sigmoid backward(10)  = 4.5395807735907655e-05
sigmoid backward(1)   = 0.19661193324148185
sigmoid backward(0.5) = 0.2350037122015945


Perhatikan bahwa makin kecil input, makin kecil gradiennya

Dan hal ini makin memburuk jika kita memiliki banyak layer. Dikarenakan jika kita memiliki matrik gradien yang berisi nilai desimal, dan saat propagasi balik kita kalikan gradien dengan gradien layer yang juga berisi nilai desimal, maka perkalian tersebut tentunya akan menghasilkan nilai desimal yang lebih kecil lagi. 

<img src="https://image.ibb.co/jSc4ve/gradien.jpg" alt="gradien"/>

Jika nilai tersebut kita bawa hingga ke layer pertama, maka sisa nilai gradien di ujung layer akan limit ke nilai 0, atau bahkan karena keterbatasan bit penyimpanan di komputer menyebabkan gradien menjadi 0. Padahal gradien adalah nilai yang harus ditambahkan ke matrik bobot agar Jaringan belajar


### Vanishing Gradient di Jaringan Saraf Tiruan

Sekarang, mari kita coba lihat apa yang terjadi pada gradien jika kita lakukan pelatihan pada suatu Jaringan Saraf Tiruan dengan 10 Layer

Mula-mula kita bangkitkan sebuah data sebanyak 1000 data dengan jumlah atribut 500

In [0]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import minmax_scale

x, y = make_classification(n_samples=1000, n_features=500)
x = minmax_scale(x, feature_range=(-1, 1))

y = np.expand_dims(y, 1)
nfitur = x.shape[1]
nlabel = y.shape[1]

inisialisasi 10 layer jaringan saraf tiruan dengan 512 neuron di setiap layernya

In [0]:
nhidden = 512

w0 = np.random.randn(nfitur , nhidden)*.01 
w1 = np.random.randn(nhidden, nhidden)*.01
w2 = np.random.randn(nhidden, nhidden)*.01
w3 = np.random.randn(nhidden, nhidden)*.01
w4 = np.random.randn(nhidden, nhidden)*.01
w5 = np.random.randn(nhidden, nhidden)*.01
w6 = np.random.randn(nhidden, nhidden)*.01
w7 = np.random.randn(nhidden, nhidden)*.01
w8 = np.random.randn(nhidden, nlabel )*.01
b0 = np.zeros((1, nhidden))
b1 = np.zeros((1, nhidden))
b2 = np.zeros((1, nhidden))
b3 = np.zeros((1, nhidden))
b4 = np.zeros((1, nhidden))
b5 = np.zeros((1, nhidden))
b6 = np.zeros((1, nhidden))
b7 = np.zeros((1, nhidden))
b8 = np.zeros((1, nlabel))

#### Forward Pass
Mari kita coba propagasi maju pada jaringan dan lihat rata-rata aktivasi di setiap layernya

In [0]:
# proses maju
layer1, cache1 = affine_forward(x, w0, b0)
aktivasi1 = sigmoid_forward(layer1)

layer2, cache2 = affine_forward(aktivasi1, w1, b1)
aktivasi2 = sigmoid_forward(layer2)

layer3, cache3 = affine_forward(aktivasi2, w2, b2)
aktivasi3 = sigmoid_forward(layer3)

layer4, cache4 = affine_forward(aktivasi3, w3, b3)
aktivasi4 = sigmoid_forward(layer4)

layer5, cache5 = affine_forward(aktivasi4, w4, b4)
aktivasi5 = sigmoid_forward(layer5)

layer6, cache6 = affine_forward(aktivasi5, w5, b5)
aktivasi6 = sigmoid_forward(layer6)

layer7, cache7 = affine_forward(aktivasi6, w6, b6)
aktivasi7 = sigmoid_forward(layer7)

layer8, cache8 = affine_forward(aktivasi7, w7, b7)
aktivasi8 = sigmoid_forward(layer8)

layer9, cache9 = affine_forward(aktivasi8, w8, b8)
aktivasi9 = sigmoid_forward(layer9)

In [20]:
print('rata-rata aktivasi layer ke-1 = ',np.mean(np.mean(aktivasi1)))
print('rata-rata aktivasi layer ke-3 = ',np.mean(np.mean(aktivasi3)))
print('rata-rata aktivasi layer ke-5 = ',np.mean(np.mean(aktivasi5)))
print('rata-rata aktivasi layer ke-7 = ',np.mean(np.mean(aktivasi7)))
print('rata-rata aktivasi layer ke-9 = ',np.mean(np.mean(aktivasi9)))

rata-rata aktivasi layer ke-1 =  0.5000718445401491
rata-rata aktivasi layer ke-3 =  0.4992324863637339
rata-rata aktivasi layer ke-5 =  0.5013928842193753
rata-rata aktivasi layer ke-7 =  0.5003229023227391
rata-rata aktivasi layer ke-9 =  0.44782931105101204


mungkin terlihat rata-rata aktivasi tidak jauh berbeda. Hal ini dikarenakan Jaringan baru mulai dijalankan, sehingga isi bobot setiap layer masih setara

#### Backward Pass
Sekarang, mari kita lihat pada proses mundur untuk menghitung gradiennya

In [29]:
# hitung error
error = y - aktivasi9
print("mse = %0.20f" % (np.mean(error ** 2)))


#proses mundur
g_layer9 = sigmoid_backward(error, aktivasi9)
dw8, db8, g_aktivasi8 = affine_backward(g_layer9, cache9)

g_layer8 = sigmoid_backward(g_aktivasi8, aktivasi8)
dw7, db7, g_aktivasi7 = affine_backward(g_layer8, cache8)

g_layer7 = sigmoid_backward(g_aktivasi7, aktivasi7)
dw6, db6, g_aktivasi6 = affine_backward(g_layer7, cache7)

g_layer6 = sigmoid_backward(g_aktivasi6, aktivasi6)
dw5, db5, g_aktivasi5 = affine_backward(g_layer6, cache6)

g_layer5 = sigmoid_backward(g_aktivasi5, aktivasi5)
dw4, db4, g_aktivasi4 = affine_backward(g_layer5, cache5)

g_layer4 = sigmoid_backward(g_aktivasi4, aktivasi4)
dw3, db3, g_aktivasi3 = affine_backward(g_layer4, cache4)

g_layer3 = sigmoid_backward(g_aktivasi3, aktivasi3)
dw2, db2, g_aktivasi2 = affine_backward(g_layer3, cache3)

g_layer2 = sigmoid_backward(g_aktivasi2, aktivasi2)
dw1, db1, g_aktivasi1 = affine_backward(g_layer2, cache2)

g_layer1 = sigmoid_backward(g_aktivasi1, aktivasi1)
dw0, db0, dx = affine_backward(g_layer1, cache1)

mse = 0.2527218


In [30]:
print('rata-rata gradien layer ke-9 = ',np.mean(np.mean(g_aktivasi8)))
print('rata-rata gradien layer ke-7 = ',np.mean(np.mean(g_aktivasi6)))
print('rata-rata gradien layer ke-5 = ',np.mean(np.mean(g_aktivasi4)))
print('rata-rata gradien layer ke-3 = ',np.mean(np.mean(g_aktivasi2)))
print('rata-rata gradien layer pe-1 = ',np.mean(np.mean(dx)))

rata-rata gradien layer ke-9 =  -1.0871507056459916e-05
rata-rata gradien layer ke-7 =  -2.038765088819387e-08
rata-rata gradien layer ke-5 =  7.390546394047515e-11
rata-rata gradien layer ke-3 =  1.0433140689723076e-13
rata-rata gradien layer pe-1 =  6.324006972723931e-16


Dapat dilihat bahwa makin ke depan, rata-rata gradien makin kecil.

Ingat, bahwa Gradien adalah nilai yang harus ditambahkan kepada bobot agar bobot belajar.
Dan gradien tidak langsung ditambahkan pada bobot, namun hanya seper-bagian kecilnya saja karena akan dikalikan dengan learning rate terlebih dahulu

<p>Copyright &copy; 2019 ADF </p>